Wav2Vec 2.0 для аудио-представлений

In [5]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import librosa
# Загрузка модели и процессора
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Загрузка аудиофайла
#audio, sr = sf.read("/content/sample-3s (1).wav", sr=16000)
audio, sr = librosa.load('/content/sample-3s (1).wav', sr=16000)
# Преобразование в формат для модели
inputs = processor(audio, sampling_rate=sr, return_tensors="pt")

# Извлечение признаков
with torch.no_grad():
    outputs = model(**inputs)

# Последний скрытый слой (контекстуализированные эмбеддинги)
audio_embeddings = outputs.last_hidden_state  # [1, seq_len, 768]
print(f"Размерность эмбеддингов: {audio_embeddings.shape}")

Размерность эмбеддингов: torch.Size([1, 159, 768])


классификация

In [9]:
audio, sr = librosa.load('/content/6880efe0845d4f1.mp3', sr=16000)
# Преобразование в формат для модели
inputs = processor(audio, sampling_rate=sr, return_tensors="pt")

In [10]:
from transformers import HubertForSequenceClassification

# Загрузка модели для классификации
model = HubertForSequenceClassification.from_pretrained("superb/hubert-base-superb-er")

# Предсказание эмоций
with torch.no_grad():
    logits = model(**inputs).logits
    predicted_class = torch.argmax(logits).item()

emotion_labels = ["нейтрально", "счастье", "грусть", "гнев", "страх"]
print(f"Предсказанная эмоция: {emotion_labels[predicted_class]}")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Предсказанная эмоция: счастье
